In [13]:
import json
import bs4
import requests
import pandas
import dryscrape
import re
import time
import os.path

In [14]:
if not os.path.exists('autotradevans.json'):
    with open('autotradevans.json', 'w') as fp:
        pass

In [22]:
def extract_posting(soup):
    posting = {}
    
    image_container = soup.find('div', {'class':'flashcontainer'}).text
    image_re = re.compile('<img>(.*?)</img>')
    images = [image for image in image_re.findall(image_container) if image]
    [s.extract() for s in soup('script')]

    for row in soup.find('div', {'class':'at_specsBox'}).find_all('div', {'class':'at_row'}):
        key = row.find('span').text.strip()
        value = row.find('div', {'class':'at_value'}).text.strip()
        posting[key] = value

    body = soup.find('span', {'class':'vehicleDescription'}).text.strip()
    title = soup.find('h1', {'id':'ctl00_ctl00_MainContent_MainContent_rptAdDetail_ctl00_adDetailControl_h1Title'}).text
    title = title.split()
    car, city, price = ' '.join(title[:-3]), title[-2], title[-1]
    price = price.replace(',', '').replace("$", '')


    posting['body'] = body
    posting['car'] = car
    posting['city'] = city
    posting['images'] = images
    posting['price'] = price
    return (posting)

In [23]:
sess = dryscrape.Session()
sess.set_attribute('auto_load_images', False)
sess.visit('http://wwwa.autotrader.ca/heavy-trucks/commercial/bc/?prx=-2&prv=British+Columbia&loc=surrey%2c+bc&sts=New-Used&pRng=10%2c14000&hprc=False&wcp=False&inMarket=advancedSearch&rcs=0&rcp=100')
time.sleep(9)
soup = bs4.BeautifulSoup(sess.body())
[s.extract() for s in soup('script')]

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[]

In [25]:
sess.body()

''

In [17]:
print (sess.body())

In [28]:
with open('autotrade_home.html') as fp:
    soup = bs4.BeautifulSoup(''.join(fp.readlines()))
    scripts = [s.extract() for s in soup('script')]
print (scripts)

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[<script async="" src="http://s0.2mdn.net/instream/video/client.js" type="text/javascript"></script>, <script src="http://traderca.demdex.net/event?d_nsid=0&amp;d_ld=_ts%3D1474913819140&amp;d_rtbd=json&amp;d_jsonv=1&amp;d_dst=1&amp;d_cb=demdexRequestCallback_0_1474913819140" type="text/javascript"></script>, <script src="http://js-agent.newrelic.com/nr-974.min.js"></script>, <script src="//imasdk.googleapis.com/js/sdkloader/ima3.js"></script>, <script async="" src="http://www.googletagservices.com/tag/js/gpt.js" type="text/javascript"></script>, <script async="" src="http://b.scorecardresearch.com/beacon.js" type="text/javascript"></script>, <script async="" src="http://stats.g.doubleclick.net/dc.js" type="text/javascript"></script>, <script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-K7JHZJ"></script>, <script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"404c1769df","applicationID":"16

In [29]:
with open('autotradevans.json') as fp:
    vans = [json.loads(line) for line in fp]
crawled_vans = set([van['url'] for van in vans])
crawled_vans

set()

In [30]:
with open('autotradevans.json') as fp:
    vans = [json.loads(line) for line in fp]
crawled_vans = set([van['url'] for van in vans])

with open('autotradevans.json', 'a') as fp:
    for listing in soup.find_all('div', {'class':'at_result'}):
        link = listing.find('div', {'class':'at_infoArea'}).find('a')
        url = link['href']
        
        if url in crawled_vans: continue
            
        van = requests.get(url)
        posting = extract_posting(bs4.BeautifulSoup(van.text))
        posting['url'] = url
        crawled_vans.add(url) 
        fp.write(json.dumps(posting) + '\n')
        time.sleep(5)

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [32]:
pandas.set_option('display.expand_frame_repr', False)
pandas.set_option('display.max_colwidth', 300)
pandas.set_option('display.width', 1000)
pandas.set_option('display.max_rows', 1000)
frame = pandas.DataFrame(crawled_vans[0]['lists'][0]['vehicles'])
frame.info()

TypeError: 'set' object does not support indexing

In [33]:
frame[['odometer', 'make', 'model', 'price']]

NameError: name 'frame' is not defined

In [ ]:
data.find('5-28552591')